In [1]:
#hyperparameter
Timedegression = 0.995 
CostOfMatch = 0.05

In [2]:
#Create class for save object on horse , jockey 
class Horse:
    def __init__(self, HorseID,HorseName,first_match_date,last_match_date):
        self.HorseID = HorseID
        self.HorseName = HorseName
        self.ParticipateCount = 0
        self.first_match_date = first_match_date
        self.last_match_date = last_match_date
        self.Score = 1000

class Jockey:
    JockeyID = 0  # Class-level variable to store the current ID
    def __init__(self, JockeyName, first_match_date, last_match_date):
        Jockey.JockeyID += 1  # Increment the class-level ID for the next object
        self.JockeyID = Jockey.JockeyID  # Assign the current ID to the instance
        self.JockeyName = JockeyName
        self.ParticipateCount = 0
        self.first_match_date = first_match_date
        self.last_match_date = last_match_date
        self.Score = 1000

        

In [3]:
import pandas as pd 
import numpy as np

In [4]:
#Load Data from CSV
RaceInformation = pd.read_csv(r"C:\Users\guojiakai\Desktop\HKJC\RaceInformation.csv")
RacePlaceData = pd.read_excel(r"C:\Users\guojiakai\Desktop\HKJC\output.xlsx")

In [5]:
#Filter Out that are not in Class 1-5
RacePlaceData =pd.merge(RacePlaceData, RaceInformation, on=['Date', 'Race'], how='inner')
RacePlaceData = RacePlaceData.sort_values(by=["Date", "Race"])
# Add speed M/s in Dataframe
RacePlaceData["M/s"]=RacePlaceData['Distance'].str.extract('(\d+)', expand=False).astype(float)/RacePlaceData["Finish Time"]


In [7]:
RacePlaceData["Date"]

0        01/01/2010
1        01/01/2010
2        01/01/2010
3        01/01/2010
4        01/01/2010
            ...    
98999    31/10/2021
99000    31/10/2021
99001    31/10/2021
99002    31/10/2021
99003    31/10/2021
Name: Date, Length: 113402, dtype: object

In [8]:
#Change DateType
RacePlaceData["Date"] = pd.to_datetime(RacePlaceData["Date"], format='%d/%m/%Y')

Timestamp('2023-04-09 00:00:00')

In [10]:
#Create  dictionary to Store Jockey and Horse Object
HorseList ={}
JockeyList ={}
#Create Date for looping 
Dates = sorted(pd.to_datetime(RacePlaceData["Date"].unique()))

In [11]:
#Loop of each Race Day
for Date in Dates:
    for HorseID in HorseList : #loop to deduct time value 
        HorseList[HorseID].Score*Timedegression
    for JockeyName in JockeyList : #loop to deduct time value 
        JockeyList[JockeyName].Score*Timedegression   
    for race in RacePlaceData[RacePlaceData["Date"] == Date]['Race'].unique() : #Pool for each race
        pool_Horse = 0
        pool_Jockey = 0
        JockeyPlace1Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 1)]["Jockey"].values
        Place1Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 1)]["Horse ID"].values
        Place1Time = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 1)]["Finish Time"].unique()
        JockeyPlace2Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 2)]["Jockey"].values
        Place2Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 2)]["Horse ID"].values
        Place2Time  = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 2)]["Finish Time"].unique()
        JockeyPlace3Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 3)]["Jockey"].values
        Place3Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 3)]["Horse ID"].values
        Place3Time = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 3)]["Finish Time"].unique()
        JockeyPlace4Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 4)]["Jockey"].values
        Place4Index = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 4)]["Horse ID"].values
        Place4Time = RacePlaceData.loc[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race) & (RacePlaceData['Pla.'] == 4)]["Finish Time"].unique()
        
        for index,result in RacePlaceData[(RacePlaceData["Date"] == Date) & (RacePlaceData["Race"] == race)].iterrows():
            if result["Jockey"] not in JockeyList :    #Create Horse object
                JockeyList[result["Jockey"]] = Jockey(result["Jockey"],Date,Date)
            if result["Horse ID"] not in HorseList :    #Create Horse object
                HorseList[result["Horse ID"]] = Horse(result["Horse ID"],result["Horse Name"],Date,Date)
            RacePlaceData.loc[index,'Participate'] = HorseList[result["Horse ID"]].ParticipateCount
            RacePlaceData.loc[index,'HorseScore'] = HorseList[result["Horse ID"]].Score
            RacePlaceData.loc[index,'JockeyParticipate'] = JockeyList[result["Jockey"]].ParticipateCount
            RacePlaceData.loc[index,'JockeyScore'] = JockeyList[result["Jockey"]].Score
            
            HorseList[result["Horse ID"]].ParticipateCount = HorseList[result["Horse ID"]].ParticipateCount +1 #Participate Count +1
            JockeyList[result["Jockey"]].ParticipateCount = JockeyList[result["Jockey"]].ParticipateCount +1 #JockeyParticipate Count +1
            pool_Horse  =   pool_Horse + HorseList[result["Horse ID"]].Score *CostOfMatch   #Transfer Score to pool
            pool_Jockey =   pool_Jockey + JockeyList[result["Jockey"]].Score *CostOfMatch
            HorseList[result["Horse ID"]].Score = HorseList[result["Horse ID"]].Score *(1-CostOfMatch)         
            JockeyList[result["Jockey"]].Score = JockeyList[result["Jockey"]].Score *(1-CostOfMatch)
            HorseList[result["Horse ID"]].last_match_date = Date
        try:
            if  Place4Time-Place1Time < 0.1 :
                HorseList[Place1Index[0]].Score += pool_Horse*0.25
                HorseList[Place2Index[0]].Score += pool_Horse*0.25
                HorseList[Place3Index[0]].Score += pool_Horse*0.25
                HorseList[Place4Index[0]].Score += pool_Horse*0.25
            elif  Place3Time-Place1Time < 0.1 :
                HorseList[Place1Index[0]].Score += pool_Horse*0.3
                HorseList[Place2Index[0]].Score += pool_Horse*0.3
                HorseList[Place3Index[0]].Score += pool_Horse*0.3
                HorseList[Place4Index[0]].Score += pool_Horse*0.1
            elif  Place2Time-Place1Time < 0.1 :
                HorseList[Place1Index[0]].Score += pool_Horse*0.35
                HorseList[Place2Index[0]].Score += pool_Horse*0.35
                HorseList[Place3Index[0]].Score += pool_Horse*0.2
                HorseList[Place4Index[0]].Score += pool_Horse*0.1     
            else  :
                HorseList[Place1Index[0]].Score += pool_Horse*0.4
                HorseList[Place2Index[0]].Score += pool_Horse*0.3
                HorseList[Place3Index[0]].Score += pool_Horse*0.2
                HorseList[Place4Index[0]].Score += pool_Horse*0.1      
            if  Place4Time-Place1Time < 0.1 :
                JockeyList[JockeyPlace1Index[0]].Score += pool_Jockey*0.25
                JockeyList[JockeyPlace2Index[0]].Score += pool_Jockey*0.25
                JockeyList[JockeyPlace3Index[0]].Score += pool_Jockey*0.25
                JockeyList[JockeyPlace4Index[0]].Score += pool_Jockey*0.25
            elif  Place3Time-Place1Time < 0.1 :
                JockeyList[JockeyPlace1Index[0]].Score += pool_Jockey*0.3
                JockeyList[JockeyPlace2Index[0]].Score += pool_Jockey*0.3
                JockeyList[JockeyPlace3Index[0]].Score += pool_Jockey*0.3
                JockeyList[JockeyPlace4Index[0]].Score += pool_Jockey*0.1
            elif  Place2Time-Place1Time < 0.1 :
                JockeyList[JockeyPlace1Index[0]].Score += pool_Jockey*0.35
                JockeyList[JockeyPlace2Index[0]].Score += pool_Jockey*0.35
                JockeyList[JockeyPlace3Index[0]].Score += pool_Jockey*0.2
                JockeyList[JockeyPlace4Index[0]].Score += pool_Jockey*0.1     
            else  :
                JockeyList[JockeyPlace1Index[0]].Score += pool_Jockey*0.4
                JockeyList[JockeyPlace2Index[0]].Score += pool_Jockey*0.3
                JockeyList[JockeyPlace3Index[0]].Score += pool_Jockey*0.2
                JockeyList[JockeyPlace4Index[0]].Score += pool_Jockey*0.1              
        except IndexError:
            pass
                

C:\Users\guojiakai\AppData\Local\Temp\ipykernel_7296\578934058.py:41: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if  Place4Time-Place1Time < 0.1 :
C:\Users\guojiakai\AppData\Local\Temp\ipykernel_7296\578934058.py:51: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif  Place2Time-Place1Time < 0.1 :
C:\Users\guojiakai\AppData\Local\Temp\ipykernel_7296\578934058.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  elif  Place3Time-Place1Time < 0.1 :


In [16]:
RacePlaceData=RacePlaceData[['Date','Race','Pla.','Horse Name','Horse ID','Jockey','Win Odds','Finish Time',\
               'M/s','Participate','HorseScore','JockeyParticipate','JockeyScore']]

In [17]:
HorseStatus = pd.DataFrame(columns=['Horse ID', 'Horse Name', ' Horse Participate Count',"First Match","Last Match","Horse Score"])
i=1
for horse in HorseList:
    HorseStatus.loc[i]=[HorseList[horse].HorseID,HorseList[horse].HorseName,\
                        HorseList[horse].ParticipateCount,HorseList[horse].first_match_date,\
                        HorseList[horse].last_match_date,HorseList[horse].Score]
    i += 1

In [18]:
JockeyStatus = pd.DataFrame(columns=['Jockey ID', 'Jockey Name', 'Jockey Participate Count',\
                                     "First Match","Last Match"," Jockey Score"])
i=1
for Jockey in JockeyList:
    JockeyStatus.loc[i]=[JockeyList[Jockey].JockeyID,JockeyList[Jockey].JockeyName,\
                        JockeyList[Jockey].ParticipateCount,JockeyList[Jockey].first_match_date,\
                        JockeyList[Jockey].last_match_date,JockeyList[Jockey].Score]
    i += 1

In [19]:
writer = pd.ExcelWriter(r"C:\Users\guojiakai\Desktop\HKJC\FinalOutput.xlsx")
JockeyStatus.to_excel(writer, sheet_name='JockeyStatus', index=False)
HorseStatus.to_excel(writer, sheet_name='HorseStatus', index=False)
RacePlaceData.to_excel(writer, sheet_name='RacePlaceData', index=False)
writer.save()

C:\Users\guojiakai\AppData\Local\Temp\ipykernel_7296\3607406333.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
